<h1>IBM Applied Data Science Capstone Course by Coursera<h1>
<h3>Opening a New Shopping Mall in Mumbai, India<h3>

__1. Import libraries__

In [2]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [4]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


folium-0.5.0         | 45 KB     | #################################

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


__2. Scrap data from Wikipedia page into a DataFrame__

In [6]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [7]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [8]:
# create a list to store neighborhood data
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [10]:
# create a new DataFrame from the list
mum_df = pd.DataFrame({"Neighborhood": neighborhoodList})
mum_df

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup
5,Borivali
6,Charkop
7,Chembur
8,Dahisar
9,Devipada


In [11]:
# print the number of rows of the dataframe
mum_df.shape

(40, 1)

__3. Get the geographical coordinates__

In [12]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [14]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in mum_df["Neighborhood"].tolist() ]

In [15]:
coords

[[19.11848309908247, 72.84177419095158],
 [19.042830000000038, 72.92734000000007],
 [19.06293000000005, 72.92666000000008],
 [19.054220000000043, 72.84019000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.062200000000075, 72.90242000000006],
 [19.250030000000038, 72.85908000000006],
 [19.224720000000048, 72.86606000000006],
 [19.212870000000066, 73.08324000000005],
 [19.00538889189226, 72.85576887678867],
 [19.086476606699875, 72.9089562772808],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.13790000000006, 72.84941000000003],
 [19.01493000000005, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21195211212422, 72.83754191243007],
 [19.131400000000042, 72.93565000000007],
 [19.127560000000074, 72.82540000000006],
 [19.064940000000036, 72.88073000000003],
 [19.21195211212422, 72.83754191243007],
 [19.048530000000028, 72.93220000000008],


In [16]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [17]:
# merge the coordinates into the original dataframe
mum_df['Latitude'] = df_coords['Latitude']
mum_df['Longitude'] = df_coords['Longitude']

In [18]:
# check the neighborhoods and the coordinates
print(mum_df.shape)
mum_df

(40, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118483,72.841774
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062930,72.926660
3,Bandra,19.054220,72.840190
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062200,72.902420
8,Dahisar,19.250030,72.859080
9,Devipada,19.224720,72.866060


In [20]:
# save the DataFrame as CSV file
mum_df.to_csv("mum.csv", index=False)

__4. Create a map of Mumbai with neighborhoods__

In [22]:
# get the coordinates of Kuala Lumpur
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India: {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India: 18.9387711, 72.8353355.


In [53]:
# create map of Toronto using latitude and longitude values
map_mum = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mum)  
    
map_mum

In [25]:
# save the map as HTML file
map_mum.save('map_mum.html')

__5. Use the Foursquare API to explore the neighborhoods__

In [26]:
# define Foursquare Credentials and Version
CLIENT_ID = '11JCCHCAKSXOS33JFKGQHMEQSTHS2R2JPKRMXD1H1GZBYEVZ' # your Foursquare ID
CLIENT_SECRET = 'MKVTOHBYOIGOWZNHHGDTN0C32WN14OS54UMJVAVUMNVUU5NS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 11JCCHCAKSXOS33JFKGQHMEQSTHS2R2JPKRMXD1H1GZBYEVZ
CLIENT_SECRET:MKVTOHBYOIGOWZNHHGDTN0C32WN14OS54UMJVAVUMNVUU5NS


In [27]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [28]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2669, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118483,72.841774,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118483,72.841774,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118483,72.841774,Naturals,19.111204,72.837255,Ice Cream Shop
3,Andheri,19.118483,72.841774,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
4,Andheri,19.118483,72.841774,Starbucks Coffee : A Tata Alliance,19.114569,72.836205,Coffee Shop


In [29]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,19,19,19,19,19,19
Baiganwadi,10,10,10,10,10,10
Bandra,100,100,100,100,100,100
Bhandup,26,26,26,26,26,26
Borivali,100,100,100,100,100,100
Charkop,59,59,59,59,59,59
Chembur,98,98,98,98,98,98
Dahisar,60,60,60,60,60,60


In [30]:
print('Uniques Categories: {}'.format(len(venues_df['VenueCategory'].unique())))

Uniques Categories: 203


In [31]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Indian Restaurant', 'Ice Cream Shop',
       'Falafel Restaurant', 'Coffee Shop', 'Sandwich Place', 'Pub',
       'Fast Food Restaurant', 'Pizza Place', 'Juice Bar', 'Multiplex',
       'Snack Place', 'Café', 'Breakfast Spot', 'Seafood Restaurant',
       'Cocktail Bar', 'Chinese Restaurant', 'American Restaurant', 'Bar',
       'Gym / Fitness Center', 'Electronics Store', 'Department Store',
       'Diner', 'BBQ Joint', 'Asian Restaurant', 'Lounge', 'Spa',
       "Women's Store", 'Liquor Store',
       'Residential Building (Apartment / Condo)',
       'Vegetarian / Vegan Restaurant', 'Food Truck', 'Smoke Shop',
       'Park', 'Fish Market', 'Martial Arts Dojo', 'Athletics & Sports',
       'Hotel', 'Tea Room', 'Italian Restaurant', 'Food', 'Plaza',
       'Bus Station', 'Mughlai Restaurant', 'Platform',
       'Paper / Office Supplies Store', 'Food & Drink Shop',
       'Music Venue', 'Hot Dog Joint', 'Tunnel'], dtype=object)

In [32]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

__6. Analyze Each Neighborhood__

In [34]:
# one hot encoding
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mum_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]

print(mum_onehot.shape)
mum_onehot.head()

(2669, 204)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Campground,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lingerie Store,Liquor Store,Lounge,Luggage Store,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [35]:
mum_grouped = mum_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mum_grouped.shape)
mum_grouped

(40, 204)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Campground,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lingerie Store,Liquor Store,Lounge,Luggage Store,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0.00,0.000000,0.0000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.020000,0.010000,0.010000,0.00,0.020000,0.0000,0.040000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0000,0.050000,0.00,0.00,0.050000,0.000000,0.01,0.020000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.010000,0.000000,0.0,0.0000,0.010000,0.00,0.000000,0.010000,0.00,0.00,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.010000,0.000000,0.040000,0.170000,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.030000,0.00,0.00,0.010000,0.030000,0.000000,0.000000,0.000000,0.01,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.

In [36]:
len(mum_grouped[mum_grouped["Shopping Mall"] > 0])

16

In [37]:
mum_mall = mum_grouped[["Neighborhoods","Shopping Mall"]]

In [38]:
mum_mall.head()

,Neighborhoods,Shopping Mall
0,Andheri,0.000000
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.000000
4,Bhandup,0.038462


__7. Cluster Neighborhoods__

In [39]:
# set number of clusters
kclusters = 3

mum_clustering = mum_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 2, 0, 2, 0, 2], dtype=int32)

In [40]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mum_merged = mum_mall.copy()

# add clustering labels
mum_merged["Cluster Labels"] = kmeans.labels_

In [41]:
mum_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Andheri,0.000000,0
1,Anushakti Nagar,0.000000,0
2,Baiganwadi,0.000000,0
3,Bandra,0.000000,0
4,Bhandup,0.038462,1


In [42]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mum_merged = mum_merged.join(mum_df.set_index("Neighborhood"), on="Neighborhood")

print(mum_merged.shape)
mum_merged.head() # check the last columns!

(40, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118483,72.841774
1,Anushakti Nagar,0.000000,0,19.042830,72.927340
2,Baiganwadi,0.000000,0,19.062930,72.926660
3,Bandra,0.000000,0,19.054220,72.840190
4,Bhandup,0.038462,1,19.145560,72.948560


In [43]:
# sort the results by Cluster Labels
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged

(40, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118483,72.841774
24,"Matharpacady, Mumbai",0.000000,0,19.044920,72.867205
23,Mankhurd,0.000000,0,19.048530,72.932200
22,Mahavir Nagar (Kandivali),0.000000,0,19.211952,72.837542
21,Kurla,0.000000,0,19.064940,72.880730
20,Kausa,0.000000,0,19.127560,72.825400
28,Mumbra,0.000000,0,19.190560,73.022670
18,Kandivali,0.000000,0,19.211952,72.837542
17,Kalyan,0.000000,0,18.953937,72.820367
16,Juhu,0.000000,0,19.014930,72.845220


In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [45]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

__8. Examine Clusters__

In [46]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.0,0,19.118483,72.841774
24,"Matharpacady, Mumbai",0.0,0,19.044920,72.867205
23,Mankhurd,0.0,0,19.048530,72.932200
22,Mahavir Nagar (Kandivali),0.0,0,19.211952,72.837542
21,Kurla,0.0,0,19.064940,72.880730
20,Kausa,0.0,0,19.127560,72.825400
28,Mumbra,0.0,0,19.190560,73.022670
18,Kandivali,0.0,0,19.211952,72.837542
17,Kalyan,0.0,0,18.953937,72.820367
16,Juhu,0.0,0,19.014930,72.845220


In [48]:
mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
33,Thakur village,0.035088,1,19.21020,72.87541
35,Vashi,0.050000,1,19.08470,72.90484
36,Vikhroli,0.030000,1,19.11109,72.92781
19,Kanjurmarg,0.032258,1,19.13140,72.93565
4,Bhandup,0.038462,1,19.14556,72.94856
27,Mulund,0.038961,1,19.17185,72.95564


In [50]:
mum_merged.loc[mum_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
38,Western Suburbs (Mumbai),0.011628,2,19.197000,72.827630
29,Pestom sagar,0.025641,2,19.070630,72.902200
30,Seven Bungalows,0.010000,2,19.128560,72.820850
13,Goregaon,0.020000,2,19.164550,72.849460
12,Ghatkopar,0.025000,2,19.086477,72.908956
9,Devipada,0.010753,2,19.224720,72.866060
7,Chembur,0.010204,2,19.062200,72.902420
5,Borivali,0.010000,2,19.229360,72.857510
37,Wadala,0.021053,2,19.017160,72.858130
39,Worli,0.020000,2,19.007450,72.816910
